# Setup

In [ ]:
%run CvCmdApi.py

# Demo

In [ ]:
from ipywidgets import widgets
gimbal_coordinate_x_Slider = widgets.IntSlider(min=0,max=640,step=1,description='coordinate_x:',value=0)
gimbal_coordinate_y_Slider = widgets.IntSlider(min=0,max=480,step=1,description='coordinate_y:',value=0)
chassis_speed_x_Slider = widgets.FloatSlider(min=-1,max=1,step=0.01,description='speed_x:',value=0)
chassis_speed_y_Slider = widgets.FloatSlider(min=-1,max=1,step=0.01,description='speed_y:',value=0)
display(gimbal_coordinate_x_Slider)
display(gimbal_coordinate_y_Slider)
display(chassis_speed_x_Slider)
display(chassis_speed_y_Slider)

In [ ]:
oldflags = (False, False, False)
while True:
    flags = CvCmder.CvCmd_Heartbeat(gimbal_coordinate_x_Slider.value, gimbal_coordinate_y_Slider.value, chassis_speed_x_Slider.value, chassis_speed_y_Slider.value)
    if flags != oldflags:
        oldflags = flags
        print(flags)

In [ ]:
CvCmder.ser.close()

In [ ]:
CvCmder.ser.open()

In [ ]:
CvCmder.ser.readall()

In [ ]:
CvCmder.CvCmd_Reset()

## Unit test

In [ ]:
CvCmder.ser.write(CvCmder.txAckMsg)

In [ ]:
(gimbal_coordinate_x, gimbal_coordinate_y, chassis_speed_x, chassis_speed_y) = (300, 400, 1, 1)
CvCmder.txCvCmdMsg[CvCmder.DATA_PAYLOAD_INDEX:CvCmder.DATA_PAYLOAD_INDEX+12] = b''.join([gimbal_coordinate_x.to_bytes(2, 'little'), gimbal_coordinate_y.to_bytes(2, 'little'), struct.pack('<f', chassis_speed_x), struct.pack('<f', chassis_speed_y)])
# test min separation time
CvCmder.ser.write(CvCmder.txAckMsg)
while True:
    CvCmder.ser.write(CvCmder.txCvCmdMsg)
    time.sleep(0.002)

In [ ]:
# incomplete/invalid msg
CvCmder.ser.write(CvCmder.txAckMsg[:10])
CvCmder.ser.write(CvCmder.txAckMsg[5:])

In [ ]:
# Find corrupted package
stop = False
while ~stop:
    bytesRead = CvCmder.ser.read(CvCmder.ser.in_waiting)
    dataPackets = re.findall(CvCmder.eSepChar.CHAR_STX.value + CvCmder.eMsgType.MSG_MODE_CONTROL.value + b"." + CvCmder.eSepChar.CHAR_UNUSED.value + b"{11}" + CvCmder.eSepChar.CHAR_ETX.value, bytesRead)
    for i in dataPackets:
        if i !=b'\x02\x10\x02\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x03':
            print(dataPackets)
            stop = True
            break